# Analysis & Visualization of Covid19 World Data (Interactive Dashboard)

COVID-19 Data repository by John Hopkins CSSE: https://github.com/CSSEGISandData/COVID-19
(There might be delays in data updates hence slight differences in results)
<img src="COVID-19.png" width=1000 height=300 />

In [148]:
from __future__ import print_function

import pandas as pd
import numpy as np


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import Image


import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import plotly.graph_objects as go
plt.style.use('ggplot')

In [149]:
# loading data right from the source:

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')


### Confirmed Cases Data

In [150]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,876,880,898,916,933,946,948,949,964,969
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,6067,6253,6442,6629,6821,7019,7201,7377,7542,7728
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,758,760,761,761,761,761,761,761,762,762
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,45,45,48,48,48,48,50,52,52,58


### Confirmed Deaths Data

In [151]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,127,132,136,153,168,169,173,178,187,193
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,31,31,31,31,31,31,31,31,31,31
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,515,522,529,536,542,548,555,561,568,575
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,48,49,49,49,51,51,51,51,51,51
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,3,3,3,3


### Recoveries Data

In [152]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,610,648,691,745,745,778,801,850,930,938
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,682,688,694,705,714,715,727,742,758,771
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2998,3058,3158,3271,3409,3507,3625,3746,3968,4062
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,568,576,596,604,615,617,624,628,639,639
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,13,14,14,17,17,17,17,17,17,17


In [153]:
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [154]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [155]:
print('Total confirmed cases: ',confirmed_total)
print('\n')
print('Total recoveries: ',recovered_total)
print('\n')
print('Total deaths: ',deaths_total)
print('\n')
print('Total active cases: ',active_total)

Total confirmed cases:  5168433


Total recoveries:  1990963


Total deaths:  335936


Total active cases:  2836301


In [156]:
#function to highlight required columns
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = p
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1

### Top confirmed cases by country; input number of countries you want displayed

In [157]:
#function to display number of worst hit countries you want to see
def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed',ascending= False).reset_index(drop=True).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

### Worst-hit countries in terms of cases; use slider to change the number of countries you want on display

In [158]:
sorted_country_df=country_df.sort_values('confirmed',ascending=False)

In [159]:
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries (Cases)",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 900
    )
    fig.show()

interact(bubble_chart, n=10)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart(n)>

### Countries with the highest number of deaths

In [160]:
sorted_country_df.sort_values('deaths',ascending=False,inplace=True)
sorted_country_df=sorted_country_df.reset_index(drop=True)

In [161]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Countries with highest number of deaths",
    color_discrete_sequence=["maroon"], 
    height=500,
    width=1000
)

### Countries with the highest death rates

In [162]:
sorted_country_df['death_rates']=sorted_country_df['deaths']/sorted_country_df['confirmed']
sorted_country_df.sort_values('death_rates',ascending=False,inplace=True)
sorted_country_df=sorted_country_df.reset_index(drop=True)

In [168]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "death_rates",
    title= "Highest Death Rates (Total Deaths:Total Confirmed)",
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)

### Confirmed cases & deaths by day for a select country; input country you want to view

In [169]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 22:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 22:].columns))
            y_data = np.sum(np.asarray(df[df['country'].str.lower() == country.lower()].iloc[:,4:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [170]:
interact(plot_cases_of_a_country,country='world')

interactive(children=(Text(value='world', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_cases_of_a_country(country)>

### Spread on the world map

In [171]:
# creating world map using Map class
world_map = folium.Map(location=[11,0], tiles='Stamen Terrain', zoom_start=2, max_zoom = 6, min_zoom = 2)
# iterate over all the rows of confirmed_df to get the lat/long
for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='blue',
        popup='Camp Muir'
    ).add_to(world_map)
    
world_map

Developed by <a href="https://www.jacsons.co.ke/">jacsons.co.ke</a>